# Leakage coefficient fit

> *This notebook estracts the leakage coefficient from the set of 4 multi-spot smFRET measurements.*

## Load FRETBursts software

In [ ]:
from fretbursts import *

In [ ]:
import os
from IPython.display import display, Math
%matplotlib inline

In [ ]:
from IPython.html.widgets import interact, interactive, fixed
from IPython.html import widgets
from IPython.display import display
from IPython.utils.traitlets import link

## Data files

In [ ]:
data_dir = './data/2013-05-15/'

In [ ]:
data_dir = os.path.abspath(data_dir) + '/'
assert os.path.exists(data_dir), "Path '%s' does not exist." % data_dir

In [ ]:
from glob import glob
file_list = sorted(glob(data_dir + '*.hdf5'))
labels = ['12d', '17d', '22d', '27d', '7d', 'DO']
files_dict = {lab: fname for lab, fname in zip(labels, file_list)}
files_dict

## 8-spot paper plot style

In [ ]:
PLOT_DIR = './figure/'

In [ ]:
import seaborn as sns
#%run -i styles/style.py
#np.set_printoptions(formatter={'float': lambda x: '%6.2f'%x})
import matplotlib as mpl

# brewer2mpl.get_map args: set name  set type  number of colors
#bmap = brewer2mpl.get_map('Set1', 'qualitative', 9)
bmap = sns.color_palette("Set1", 9)
colors = np.array(bmap)[(1,0,2,3,4,8,6,7), :]
if mpl.__version__.startswith('1.4'):
    mpl.rcParams['axes.color_cycle'] = list(colors)    
else:
    # Version 1.5 or later
    from cycler import cycler
    mpl.rcParams['axes.prop_cycle'] = cycler('color', colors)
colors_labels = ['blue', 'red', 'green', 'violet', 'orange', 'gray', 'brown', 'pink', ]
for c, cl in zip(colors, colors_labels):
    locals()[cl] = tuple(c) # assign variables with color names
sns.palplot(colors)

## Parameters

Analysis parameters:

In [ ]:
## Background fit
bg_kwargs_auto = dict(fun=bg.exp_fit,
                 time_s = 30,
                 tail_min_us = 'auto',
                 F_bg=1.7,
                 )

## Burst search
F = 6
ph_sel = Ph_sel(Dex='Dem') 
#ph_sel = Ph_sel('all') 
size_min = 80

## D-only peak fit with KDE
bandwidth=0.03
E_range_do = (-0.05, 0.1)
weights = 'size'

In [ ]:
import pandas as pd

In [ ]:
E_do = pd.DataFrame(index=['7d', '12d', '17d', 'DO'], columns=range(8))
E_do_g = pd.DataFrame(index=['7d', '12d', '17d', 'DO'], columns=range(8))
nbursts = pd.DataFrame(index=['7d', '12d', '17d', 'DO'], columns=range(8))

## Utility functions

In [ ]:
def print_fit_report(E_pr, gamma=1, leakage=0, dir_ex_t=0, math=True):
    """Print fit and standard deviation for both corrected and uncorrected E
    Returns d.E_fit.
    """
    E_corr = fretmath.correct_E_gamma_leak_dir(E_pr, gamma=gamma, leakage=leakage, dir_ex_t=dir_ex_t)
    
    E_pr_mean = E_pr.mean()*100
    E_pr_delta = (E_pr.max() - E_pr.min())*100
    
    E_corr_mean = E_corr.mean()*100
    E_corr_delta = (E_corr.max() - E_corr.min())*100
    if math:
        display(Math(r'\text{Pre}\;\gamma\quad\langle{E}_{fit}\rangle = %.1f\%% \qquad'
                     '\Delta E_{fit} = %.2f \%%' % \
                     (E_pr_mean, E_pr_delta)))
        display(Math(r'\text{Post}\;\gamma\quad\langle{E}_{fit}\rangle = %.1f\%% \qquad'
                     '\Delta E_{fit} = %.2f \%%' % \
                     (E_corr_mean, E_corr_delta)))
    else:
        print('Pre-gamma  E (delta, mean):  %.2f  %.2f' % (E_pr_mean, E_pr_delta))
        print('Post-gamma E (delta, mean):  %.2f  %.2f' % (E_corr_mean, E_corr_delta))

## 7bp sample

In [ ]:
data_id = '7d'
d7 = loader.photon_hdf5(files_dict[data_id])
d7.calc_bg(**bg_kwargs_auto)

In [ ]:
d7.burst_search(m=10, F=F, ph_sel=ph_sel)

In [ ]:
ds7 = Sel(d7, select_bursts.nd, th1=size_min)
dx = ds7

In [ ]:
## KDE Fit
E_do.loc[data_id] = bext.fit_bursts_kde_peak(dx, bandwidth=bandwidth, x_range=E_range_do,  
                                             weights=weights)

## Gaussian fit
dx.E_fitter.fit_histogram(mfit.factory_gaussian())
E_do_g.loc[data_id] = dx.E_fitter.params['center']

## D-only selection
do_s = Sel(dx, select_bursts.E, E2=0.1)
nbursts.loc[data_id] = do_s.num_bursts

In [ ]:
dplot(dx, hist_fret, weights='size', show_kde=True, show_kde_peak=True, show_fit_value=True);
plt.xlim(xmax = 0.49)
print_fit_report(E_do.loc[data_id])

In [ ]:
dplot(dx, hist_fret, weights='size', show_model=True, show_fit_value=True, fit_from='center');
plt.xlim(xmax = 0.49)
print_fit_report(E_do_g.loc[data_id])

### Alternative plots

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(12, 8), sharex=True, sharey=True)
fig.subplots_adjust(left=0.08, right=0.96, top=0.93, bottom=0.07,
                    wspace=0.06, hspace=0.18)

for ich, ax in enumerate(axes.ravel()):
    mfit.plot_mfit(dx.E_fitter, ich=ich, ax=ax, plot_model=False, plot_kde=True)
plt.xlim(-0.2, 0.3)
print_fit_report(E_do.loc[data_id])

In [ ]:
fig, axes = plt.subplots(4, 2, figsize=(12, 8), sharex=True, sharey=True)
fig.subplots_adjust(left=0.08, right=0.96, top=0.93, bottom=0.07,
                    wspace=0.06, hspace=0.18)

for ich, ax in enumerate(axes.ravel()):
    mfit.plot_mfit(dx.E_fitter, ich=ich, ax=ax)
plt.xlim(-0.2, 0.3)
print_fit_report(E_do_g.loc[data_id])

## 12bp sample

In [ ]:
data_id = '12d'
d12 = loader.photon_hdf5(files_dict[data_id])
d12.calc_bg(**bg_kwargs_auto)

In [ ]:
d12.burst_search(m=10, F=F, ph_sel=ph_sel)

In [ ]:
ds12 = Sel(d12, select_bursts.nd, th1=size_min)
dx = ds12

In [ ]:
## KDE Fit
E_do.loc[data_id] = bext.fit_bursts_kde_peak(dx, bandwidth=bandwidth, x_range=E_range_do,  
                                             weights=weights)

## Gaussian fit
dx.E_fitter.fit_histogram(mfit.factory_gaussian())
E_do_g.loc[data_id] = dx.E_fitter.params['center']

## D-only selection
do_s = Sel(dx, select_bursts.E, E2=0.1)
nbursts.loc[data_id] = do_s.num_bursts

In [ ]:
dplot(dx, hist_fret, weights='size', show_kde=True, show_kde_peak=True, show_fit_value=True);
plt.xlim(xmax = 0.49)
print_fit_report(E_do.loc[data_id])

In [ ]:
dplot(dx, hist_fret, weights='size', show_model=True, show_fit_value=True, fit_from='center');
plt.xlim(xmax = 0.49)
print_fit_report(E_do_g.loc[data_id])

## 17bp sample

In [ ]:
data_id = '17d'
d17 = loader.photon_hdf5(files_dict[data_id])
d17.calc_bg(**bg_kwargs_auto)

In [ ]:
d17.burst_search(m=10, F=F, ph_sel=ph_sel)

In [ ]:
ds17 = Sel(d17, select_bursts.nd, th1=size_min)
dx = ds17

In [ ]:
## KDE Fit
E_do.loc[data_id] = bext.fit_bursts_kde_peak(dx, bandwidth=bandwidth, x_range=E_range_do,  
                                             weights=weights)

## Gaussian fit
dx.E_fitter.fit_histogram(mfit.factory_two_gaussians(p1_center=0.03, p2_center=0.25))
E_do_g.loc[data_id] = dx.E_fitter.params['p1_center']

## D-only selection
do_s = Sel(dx, select_bursts.E, E2=0.1)
nbursts.loc[data_id] = do_s.num_bursts

In [ ]:
dplot(dx, hist_fret, weights='size', show_kde=True, show_kde_peak=True, show_fit_value=True);
plt.xlim(xmax = 0.49)
print_fit_report(E_do.loc[data_id])

In [ ]:
dplot(dx, hist_fret, weights='size', show_model=True, show_fit_value=True, fit_from='p1_center');
plt.xlim(xmax = 0.49)
print_fit_report(E_do_g.loc[data_id])

## DO sample

In [ ]:
data_id = 'DO'
do = loader.photon_hdf5(files_dict[data_id])
do.calc_bg(**bg_kwargs_auto)

In [ ]:
do.burst_search(L=10, m=10, F=F, ph_sel=ph_sel)

In [ ]:
dos = Sel(do, select_bursts.nd, th1=size_min)
dx = dos

In [ ]:
## KDE Fit
E_do.loc[data_id] = bext.fit_bursts_kde_peak(dx, bandwidth=bandwidth, x_range=E_range_do,  
                                             weights=weights)

## Gaussian fit
dx.E_fitter.fit_histogram(mfit.factory_gaussian())
E_do_g.loc[data_id] = dx.E_fitter.params['center']

## D-only selection
nbursts.loc[data_id] = dx.num_bursts

In [ ]:
dplot(dx, hist_fret, weights='size', show_kde=True, show_kde_peak=True, show_fit_value=True);
plt.xlim(xmax = 0.49)
print_fit_report(E_do.loc[data_id])

In [ ]:
dplot(dx, hist_fret, weights='size', show_model=True, show_fit_value=True, fit_from='center');
plt.xlim(xmax = 0.49)
print_fit_report(E_do_g.loc[data_id])

# Results

In [ ]:
ph_sel

In [ ]:
E_do_kde = E_do.copy()

In [ ]:
E_do = E_do_kde

In [ ]:
leakage = E_do / (1 - E_do)
leakage

In [ ]:
leakage.T.plot();

## Average leakage

### Mean per sample:

In [ ]:
lk_s = pd.DataFrame(index=['mean', 'std'], columns=E_do.index)

lk_s.loc['mean'] = leakage.mean(1)*100
lk_s.loc['std'] = leakage.std(1)*100

lk_s['mean'] = lk_s.mean(1)
lk_s

In [ ]:
lk_s.loc['mean'].plot()
ylim(2, 4)

### Mean per sample (weighted on the number of bursts):

In [ ]:
nbursts

In [ ]:
lk_sw = pd.DataFrame(index=['mean', 'std'], columns=E_do.index)

lk_sw.loc['mean'] = (nbursts*leakage).sum(1)/nbursts.sum(1)*100
lk_sw.loc['std'] = (nbursts*leakage).std(1)/nbursts.sum(1)*100

lk_sw['mean'] = (nbursts.sum(1)*lk_sw).sum(1)/nbursts.sum(1).sum()
lk_sw

In [ ]:
lk_sw.loc['mean'].plot()
ylim(2, 4)

### Mean per channel:

In [ ]:
lk_c = pd.DataFrame(index=['mean', 'std'], columns=E_do.columns)

lk_c.loc['mean'] = leakage.mean()*100
lk_c.loc['std'] = leakage.std()*100

lk_c['mean'] = lk_c.mean(1)
lk_c

In [ ]:
lk_c.loc['mean'].plot()
ylim(2, 4)

### Mean per channel (weighted on the number of bursts):

In [ ]:
lk_cw = pd.DataFrame(index=['mean', 'std'], columns=E_do.columns)

lk_cw.loc['mean'] = (nbursts*leakage).sum()/nbursts.sum()*100
lk_cw.loc['std'] = (nbursts*leakage).std()/nbursts.sum()*100

lk_cw['mean'] = lk_cw.mean(1)
lk_cw

In [ ]:
lk_cw.loc['mean'].plot()
ylim(2, 4)

In [ ]:
mch_plot_bg(d7)

> **NOTE:** There is a per-channel trend that cannot be ascribed to the background 
> because we performend a D-emission burst search and selection and the leakage vs ch
> does not resemble the D-background vs channel curve.
>
> The effect is probably due to slight PDE variations that change the $\gamma$ 
> on a per-spot basis.

### Weighted mean of the weighted mean

In [ ]:
lk = (lk_cw.ix['mean', :-1]*nbursts.mean()).sum()/(nbursts.mean().sum())/100
lk_2 = (lk_sw.ix['mean', :-1]*nbursts.mean(1)).sum()/(nbursts.mean(1).sum())/100

assert np.allclose(lk, lk_2)

print('Mean leakage: ', lk)

## Save results

In [ ]:
if ph_sel == Ph_sel('all'):
    bsearch_str = 'all-ph'
elif ph_sel == Ph_sel(Dex='Dem'):
    bsearch_str = 'Dem'

### Per-channel

In [ ]:
lk_cw.to_csv('results/Multi-spot - leakage coefficient mean per-ch %s.txt' % bsearch_str)

In [ ]:
pd.read_csv('results/Multi-spot - leakage coefficient mean per-ch %s.txt' % bsearch_str, 
            index_col=0)

### Per-sample

In [ ]:
lk_sw.to_csv('results/Multi-spot - leakage coefficient mean per-sample %s.txt' % bsearch_str)

In [ ]:
pd.read_csv('results/Multi-spot - leakage coefficient mean per-sample %s.txt' % bsearch_str, 
            index_col=0)

### Mean

In [ ]:
with open('results/Multi-spot - leakage coefficient %s.txt' % bsearch_str, 'w') as f:
    f.write(str(lk))

# Notebook style

In [ ]:
from IPython.core.display import HTML
HTML(open("./styles/custom.css", "r").read())